In [1]:
# futures_grid_bot_sim.py
# Full simulation pipeline — uses live data from Binance via ccxt, but DOES NOT place real trades.
# It will open LONG grids when model predicts uptrend and SHORT grids when model predicts downtrend.
# No leverage is used (placeholder comments show where to add leverage later).

import ccxt
import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
import ta
import talib
import uuid
import time
from datetime import datetime, timedelta, timezone
import math


In [2]:
from pprint import pprint

In [3]:
import ccxt

# === Connect to Binance Futures Testnet ===
exchange = ccxt.binance({
    'apiKey': 'q0HAhhxXOJr8HBbmYFYe0QMhPMrbfWVGngOUeccqXQHdgCk5fKqq97vfykJm8EGk',
    'secret': 'LSkB2x2lO8r5fx6qaTR4rGObGFjRHbJHwHPbpSY4LQ8Hp9uU7vKaueVaDavbVA3C',
    'options': {'defaultType': 'future'},
})

exchange.set_sandbox_mode(True)  # Enable Testnet mode

print("✅ Connected to Binance Futures Testnet")





✅ Connected to Binance Futures Testnet


In [4]:
symbol = 'BTC/USDT'
amount = 0.001
entry_price = 106800
take_profit = 106850
stop_loss = 106720

print("📤 Placing entry order...")
entry_order = exchange.create_order(
    symbol=symbol,
    type='LIMIT',
    side='BUY',
    amount=amount,
    price=entry_price,
    params={'timeInForce': 'GTC'}
)

print(f"✅ Entry order created: {entry_order['id']}")


📤 Placing entry order...
✅ Entry order created: 6281354168


In [5]:
"""print("⏳ Waiting for entry order to be filled...")
while True:
    order = exchange.fetch_order(entry_order['id'], symbol)
    if order['status'] == 'closed':
        print("✅ Entry order filled! Position opened.")
        break
    elif order['status'] == 'canceled':
        print("❌ Order canceled before fill.")
        break
    time.sleep(1)


if order['status'] == 'closed':
    # 🔹 Take-Profit Order
    tp_order = exchange.create_order(
        symbol=symbol,
        type='LIMIT',
        side='SELL',     # opposite of entry
        amount=amount,
        price=take_profit,
        params={'reduceOnly': True, 'timeInForce': 'GTC'}
    )
    print(f"✅ Take-Profit set at {take_profit}")

    # 🔹 Stop-Loss Order
    sl_order = exchange.create_order(
        symbol=symbol,
        type='STOP_MARKET',
        side='SELL',
        amount=amount,
        params={'stopPrice': stop_loss, 'reduceOnly': True}
    )
    print(f"✅ Stop-Loss set at {stop_loss}")
"""

'print("⏳ Waiting for entry order to be filled...")\nwhile True:\n    order = exchange.fetch_order(entry_order[\'id\'], symbol)\n    if order[\'status\'] == \'closed\':\n        print("✅ Entry order filled! Position opened.")\n        break\n    elif order[\'status\'] == \'canceled\':\n        print("❌ Order canceled before fill.")\n        break\n    time.sleep(1)\n\n\nif order[\'status\'] == \'closed\':\n    # 🔹 Take-Profit Order\n    tp_order = exchange.create_order(\n        symbol=symbol,\n        type=\'LIMIT\',\n        side=\'SELL\',     # opposite of entry\n        amount=amount,\n        price=take_profit,\n        params={\'reduceOnly\': True, \'timeInForce\': \'GTC\'}\n    )\n    print(f"✅ Take-Profit set at {take_profit}")\n\n    # 🔹 Stop-Loss Order\n    sl_order = exchange.create_order(\n        symbol=symbol,\n        type=\'STOP_MARKET\',\n        side=\'SELL\',\n        amount=amount,\n        params={\'stopPrice\': stop_loss, \'reduceOnly\': True}\n    )\n    print(f"

In [28]:
symbol = "BTC/USDT"
amount_per_order = 0.001  # per grid order
tp_offset = 2              # take profit two grids above
grid_count = 5            # number of grids
range_size = 200           # price range around current price
leverage = 10



# -------------------------------
# 2️⃣ Generate dynamic grid prices
# -------------------------------
ticker = exchange.fetch_ticker(symbol)
current_price = float(ticker['last'])

lower_price = current_price - range_size
upper_price = current_price + range_size
grid_step = (upper_price - lower_price) / (grid_count - 1)
grid_prices = [lower_price + i*grid_step for i in range(grid_count)]

print("Dynamic grids:", grid_prices)

# -------------------------------
# 3️⃣ Place buy limit orders at each grid
# -------------------------------
for price in grid_prices:
    try:
        order = exchange.create_order(
        symbol=symbol,
        type='LIMIT',
        side='BUY',
        amount=amount_per_order,
        price=price,
        params={'timeInForce': 'GTC'}
)
        print(f"Placed buy limit order at {price}, order id: {order['id']}")
        time.sleep(1)  # slight delay to avoid rate-limit issues
    except ccxt.BaseError as e:
        print(f"Error placing buy order at {price}: {e}")
        time.sleep(1)
        
all_orders = exchange.fetch_orders(symbol)
print("Total orders:", len(all_orders))

for o in all_orders:
    print(f"{o['id']} | {o['side']} | {o['status']} | price={o['price']} | filled={o['filled']}")        


"""# -------------------------------
# 4️⃣ Monitor filled orders and place TP
# -------------------------------
filled_orders_tracked = set()  # store (order_id, rounded filled_price)

while True:
    try:
        closed_orders = exchange.fetch_closed_orders(symbol)
        tp_grid_dict = {}  # tp_price -> total amount

        for order in closed_orders:
            order_id = order['id']
            filled_price = float(order['average'])
            filled_amount = float(order['amount'])

            key = (order_id, round(filled_price, 2))  # avoid duplicates from partial fills
            if order['side'] == 'buy' and key not in filled_orders_tracked:
                filled_orders_tracked.add(key)

                # Find closest grid index
                closest_grid_index = min(range(len(grid_prices)),
                                         key=lambda i: abs(grid_prices[i] - filled_price))
                tp_index = closest_grid_index + tp_offset
                if tp_index >= len(grid_prices):
                    tp_index = len(grid_prices) - 1
                tp_price = grid_prices[tp_index]

                # Fetch current market price to avoid immediate trigger
                ticker = exchange.fetch_ticker(symbol)
                market_price = float(ticker['last'])
                if tp_price <= market_price:
                    tp_price = market_price * 1.001  # tiny buffer

                # Merge multiple fills targeting same TP grid
                if tp_price not in tp_grid_dict:
                    tp_grid_dict[tp_price] = 0
                tp_grid_dict[tp_price] += filled_amount

        # Place one TP per grid price
        for tp_price, total_amount in tp_grid_dict.items():
            tp_order = exchange.create_order(
                symbol=symbol,
                type='take_profit_market',
                side='sell',
                amount=total_amount,
                params={'stopPrice': tp_price, 'reduceOnly': True}
            )
            print(f"✅ TP placed: {total_amount} BTC at {tp_price}")

        time.sleep(1.5)  # slight delay to reduce rate-limit errors

    except Exception as e:
        print("Error monitoring orders:", e)
        time.sleep(2)"""

Dynamic grids: [110979.0, 111079.0, 111179.0, 111279.0, 111379.0]
Placed buy limit order at 110979.0, order id: 6289527243
Placed buy limit order at 111079.0, order id: 6289530395
Placed buy limit order at 111179.0, order id: 6289536481
Placed buy limit order at 111279.0, order id: 6289539992
Placed buy limit order at 111379.0, order id: 6289543532
Total orders: 388
6100876212 | buy | closed | price=105667.8 | filled=0.001
6100876349 | sell | closed | price=105667.7 | filled=0.001
6114070665 | sell | closed | price=105336.8 | filled=0.002
6114071341 | sell | closed | price=105397.3 | filled=0.002
6114071610 | sell | closed | price=105457.8 | filled=0.002
6114071901 | sell | closed | price=105518.3 | filled=0.002
6114072170 | sell | closed | price=105578.8 | filled=0.002
6114072465 | sell | closed | price=105639.4 | filled=0.002
6114073619 | sell | closed | price=105699.9 | filled=0.002
6114074189 | sell | closed | price=105760.4 | filled=0.002
6114075761 | sell | closed | price=105820.

'# -------------------------------\n# 4️⃣ Monitor filled orders and place TP\n# -------------------------------\nfilled_orders_tracked = set()  # store (order_id, rounded filled_price)\n\nwhile True:\n    try:\n        closed_orders = exchange.fetch_closed_orders(symbol)\n        tp_grid_dict = {}  # tp_price -> total amount\n\n        for order in closed_orders:\n            order_id = order[\'id\']\n            filled_price = float(order[\'average\'])\n            filled_amount = float(order[\'amount\'])\n\n            key = (order_id, round(filled_price, 2))  # avoid duplicates from partial fills\n            if order[\'side\'] == \'buy\' and key not in filled_orders_tracked:\n                filled_orders_tracked.add(key)\n\n                # Find closest grid index\n                closest_grid_index = min(range(len(grid_prices)),\n                                         key=lambda i: abs(grid_prices[i] - filled_price))\n                tp_index = closest_grid_index + tp_offset\n

In [8]:
symbol = 'BTC/USDT'
order = exchange.create_order(symbol, 'limit', 'buy', 0.01, 108899)
print(order["id"])

6281816152


In [15]:
symbol = "BTC/USDT"
amount_per_order = 0.001
grid_count = 6
grid_spacing = 200  # $200 between each grid
leverage = 10

exchange.set_leverage(leverage, symbol)

# -------------------------------
# 2️⃣ Generate grid prices
# -------------------------------
ticker = exchange.fetch_ticker(symbol)
current_price = float(ticker["last"])

grid_prices = [current_price - (grid_count // 2 - i) * grid_spacing for i in range(grid_count)]
grid_prices.sort()

print("📊 Grid levels:")
for i, p in enumerate(grid_prices):
    print(f"Grid {i+1}: {p}")

# -------------------------------
# 3️⃣ Place buy orders at each grid
# -------------------------------
for i, price in enumerate(grid_prices):
    try:
        order = exchange.create_order(symbol, "limit", "buy", amount_per_order, price)
        print(f"✅ Placed BUY limit at {price}")

        # To respect rate limits
        time.sleep(0.6)

    except Exception as e:
        print(f"❌ Error placing buy at {price}: {e}")
        time.sleep(1)

# -------------------------------
# 4️⃣ Monitor fills and place sells
# -------------------------------
filled_orders_tracked = {}

print("\n🔍 Monitoring fills...")

while True:
    try:
        closed_orders = exchange.fetch_closed_orders(symbol)
        for order in closed_orders:
            order_id = order["id"]
            if order["side"] == "buy" and order_id not in filled_orders_tracked:
                filled_orders_tracked[order_id] = True
                filled_price = float(order["average"])
                filled_amount = float(order["amount"])

                # Find the next grid price for selling
                next_grid_index = min(range(len(grid_prices)),
                                      key=lambda i: abs(grid_prices[i] - filled_price)) + 1
                if next_grid_index < len(grid_prices):
                    sell_price = grid_prices[next_grid_index]
                else:
                    sell_price = filled_price + grid_spacing  # fallback

                # Place SELL limit order
                exchange.create_order(symbol, "limit", "sell", filled_amount, sell_price)
                print(f"💰 Filled BUY at {filled_price} → SELL placed at {sell_price}")

                # Delay to avoid API limit
                time.sleep(0.5)

        time.sleep(1.5)

    except Exception as e:
        print("⚠️ Error monitoring orders:", e)
        time.sleep(2)

📊 Grid levels:
Grid 1: 110585.6
Grid 2: 110785.6
Grid 3: 110985.6
Grid 4: 111185.6
Grid 5: 111385.6
Grid 6: 111585.6
✅ Placed BUY limit at 110585.6
✅ Placed BUY limit at 110785.6
✅ Placed BUY limit at 110985.6
✅ Placed BUY limit at 111185.6
✅ Placed BUY limit at 111385.6
✅ Placed BUY limit at 111585.6

🔍 Monitoring fills...
💰 Filled BUY at 105667.8 → SELL placed at 110785.6
💰 Filled BUY at 105318.5 → SELL placed at 110785.6
💰 Filled BUY at 105378.5 → SELL placed at 110785.6
💰 Filled BUY at 105438.5 → SELL placed at 110785.6
💰 Filled BUY at 105498.5 → SELL placed at 110785.6
💰 Filled BUY at 105558.5 → SELL placed at 110785.6
💰 Filled BUY at 105618.5 → SELL placed at 110785.6
💰 Filled BUY at 105678.5 → SELL placed at 110785.6
💰 Filled BUY at 105738.5 → SELL placed at 110785.6
💰 Filled BUY at 105798.5 → SELL placed at 110785.6
💰 Filled BUY at 105803.75 → SELL placed at 110785.6
💰 Filled BUY at 105803.75 → SELL placed at 110785.6
💰 Filled BUY at 105804.85 → SELL placed at 110785.6
💰 Filled

KeyboardInterrupt: 

In [18]:
# Get the current price
ticker = exchange.fetch_ticker(symbol)
current_price = float(ticker["last"])

# Create 3 limit buy orders below current price
prices = [current_price - 50, current_price - 100, current_price - 150]

order_ids = []

for price in prices:
    try:
        order = exchange.create_order(symbol, "limit", "buy", amount, price)
        print(f"✅ Created BUY order at {price}")
        order_ids.append(order["id"])
        time.sleep(0.6)
    except Exception as e:
        print(f"❌ Error placing order at {price}: {e}")


✅ Created BUY order at 111070.2
✅ Created BUY order at 111020.2
✅ Created BUY order at 110970.2


In [19]:
try:
    result = exchange.cancel_all_orders(symbol)
    print("🧹 All open orders canceled successfully.")
except Exception as e:
    print("⚠️ Error canceling all orders:", e)


🧹 All open orders canceled successfully.


In [35]:
import time
import ccxt

symbol = "BTC/USDT"
amount_per_order = 0.001  # per grid order
tp_offset = 2             # take profit two grids above
grid_count = 5            # number of grids
range_size = 200          # price range around current price
leverage = 10

exchange.set_leverage(leverage, symbol)

# -------------------------------
# 2️⃣ Generate dynamic grid prices
# -------------------------------
ticker = exchange.fetch_ticker(symbol)
current_price = float(ticker['last'])

lower_price = current_price - range_size
upper_price = current_price + range_size
grid_step = (upper_price - lower_price) / (grid_count - 1)
grid_prices = [lower_price + i*grid_step for i in range(grid_count)]

print("Dynamic grids:", grid_prices)

# -------------------------------
# 3️⃣ Place buy orders at each grid
# -------------------------------
for price in grid_prices:
    try:
        if price < current_price:
            # BELOW market: regular limit order
            order_type = 'limit'
            params = {'timeInForce': 'GTC'}
        else:
            # AT or ABOVE market: stop-limit to prevent instant execution
            order_type = 'stop_market'
            params = {'stopPrice': price, 'timeInForce': 'GTC'}

        order = exchange.create_order(
            symbol=symbol,
            type=order_type,
            side='buy',
            amount=amount_per_order,
            price=price,
            params=params
        )

        print(f"Placed buy order at {price}, order id: {order['id']}, type: {order_type}")
        time.sleep(0.5)  # slight delay to reduce rate-limit issues

    except ccxt.BaseError as e:
        print(f"Error placing buy order at {price}: {e}")
        time.sleep(1)


# -------------------------------
# 5️⃣ Monitor filled orders and place TP
# -------------------------------
filled_orders_tracked = set()  # store (order_id, rounded filled_price)

while True:
    try:
        closed_orders = exchange.fetch_closed_orders(symbol)
        tp_grid_dict = {}  # tp_price -> total amount

        for order in closed_orders:
            order_id = order['id']
            filled_price = float(order['average'])
            filled_amount = float(order['amount'])

            key = (order_id, round(filled_price, 2))
            if order['side'] == 'buy' and key not in filled_orders_tracked:
                filled_orders_tracked.add(key)

                # Find closest grid index
                closest_grid_index = min(range(len(grid_prices)),
                                         key=lambda i: abs(grid_prices[i] - filled_price))
                tp_index = closest_grid_index + tp_offset
                if tp_index >= len(grid_prices):
                    tp_index = len(grid_prices) - 1
                tp_price = grid_prices[tp_index]

                # Fetch current market price to avoid immediate trigger
                ticker = exchange.fetch_ticker(symbol)
                market_price = float(ticker['last'])
                if tp_price <= market_price:
                    tp_price = market_price * 1.001

                # Merge multiple fills targeting same TP grid
                if tp_price not in tp_grid_dict:
                    tp_grid_dict[tp_price] = 0
                tp_grid_dict[tp_price] += filled_amount

        # Place one TP per grid price
        for tp_price, total_amount in tp_grid_dict.items():
            tp_order = exchange.create_order(
                symbol=symbol,
                type='take_profit_market',
                side='sell',
                amount=total_amount,
                params={'stopPrice': tp_price, 'reduceOnly': True}
            )
            print(f"✅ TP placed: {total_amount} BTC at {tp_price}")

        time.sleep(1.5)

    except Exception as e:
        print("Error monitoring orders:", e)
        time.sleep(2)


Dynamic grids: [110859.6, 110959.6, 111059.6, 111159.6, 111259.6]
Placed buy order at 110859.6, order id: 6292624927, type: limit
Placed buy order at 110959.6, order id: 6292626514, type: limit
Error placing buy order at 111059.6: binance {"code":-2021,"msg":"Order would immediately trigger."}
Placed buy order at 111159.6, order id: 6292632653, type: stop_market
Placed buy order at 111259.6, order id: 6292634749, type: stop_market
✅ TP placed: 0.05300000000000002 BTC at 111170.6596
Error monitoring orders: binance {"code":-2021,"msg":"Order would immediately trigger."}
✅ TP placed: 0.001 BTC at 111159.6
✅ TP placed: 0.001 BTC at 111259.6
Error monitoring orders: binance {"code":-2021,"msg":"Order would immediately trigger."}


KeyboardInterrupt: 

In [49]:
symbol = "BTC/USDT"
amount = 0.001

# ---------- Fetch current market ----------
ticker = exchange.fetch_ticker(symbol)
market_price = float(ticker['last'])
print("Current market price:", market_price)

# ---------- Place a buy order below market ----------
buy_price = market_price - 50  # 1% below market
order = exchange.create_order(
    symbol=symbol,
    type='limit',
    side='buy',
    amount=amount,
    price=buy_price,
    params={'timeInForce': 'GTC'}
)
print(f"Placed BUY LIMIT at {buy_price}, order id: {order['id']}")

# ---------- Wait until filled (for demo, we fetch closed orders after 5 sec) ----------
time.sleep(5)
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break

if filled_order:
    print(f"Buy order filled at {buy_price}, placing TP...")
    filled_amount = float(filled_order['amount'])
    
    # ---------- Determine TP ----------
    tp_price = buy_price * 1.02  # 2% above buy
    
    # If TP is below market, use LIMIT; if above, use TAKE_PROFIT_MARKET
    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])
    if tp_price <= current_market:
        # Use LIMIT order
        tp_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=tp_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT TP at {tp_price}, order id: {tp_order['id']}")
    else:
        # Use TAKE_PROFIT_MARKET order
        tp_order = exchange.create_order(
            symbol=symbol,
            type='take_profit_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': tp_price, 'reduceOnly': True}
        )
        print(f"Placed SELL TAKE_PROFIT_MARKET TP at {tp_price}, order id: {tp_order['id']}")

Current market price: 111023.1
Placed BUY LIMIT at 110973.1, order id: 6299027642


In [67]:
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break
if filled_order:
    print(f"Buy order filled at {buy_price}, placing TP...")
    filled_amount = float(filled_order['amount'])
    
    # ---------- Determine TP ----------
    tp_price = buy_price * 1.02  # 2% above buy
    
    # If TP is below market, use LIMIT; if above, use TAKE_PROFIT_MARKET
    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])
    tp_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=tp_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
    print(f"Placed SELL LIMIT TP at {tp_price}, order id: {tp_order['id']}")

Buy order filled at 110981.4, placing TP...
Placed SELL LIMIT TP at 113201.02799999999, order id: 6305236971


In [41]:
import time
import ccxt

symbol = "BTC/USDT"
amount = 0.001

# ---------- Fetch current market ----------
ticker = exchange.fetch_ticker(symbol)
market_price = float(ticker['last'])
print("Current market price:", market_price)

# ---------- Place a buy order BELOW market ----------
buy_price = market_price - 50  # example: $50 below current price
order = exchange.create_order(
    symbol=symbol,
    type='limit',
    side='buy',
    amount=amount,
    price=buy_price,
    params={'timeInForce': 'GTC'}
)
print(f"Placed BUY LIMIT at {buy_price}, order id: {order['id']}")

# ---------- Wait until filled (for demo, fetch closed orders after 5 sec) ----------
time.sleep(5)
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break

if filled_order:
    filled_amount = float(filled_order['amount'])
    print(f"Buy order filled at {buy_price}, placing STOP LOSS...")

    # ---------- Determine STOP LOSS ----------
    sl_price = buy_price -50  # 2% below buy price
    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])

    # If stop loss is below current market, use LIMIT; if above, use STOP_MARKET
    if sl_price <= current_market:
        # LIMIT stop-loss
        sl_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=sl_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT STOP LOSS at {sl_price}, order id: {sl_order['id']}")
    else:
        # STOP_MARKET stop-loss
        sl_order = exchange.create_order(
            symbol=symbol,
            type='stop_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': sl_price, 'reduceOnly': True}
        )
        print(f"Placed SELL STOP_MARKET STOP LOSS at {sl_price}, order id: {sl_order['id']}")


Current market price: 111034.9
Placed BUY LIMIT at 110984.9, order id: 6297631381


In [48]:
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break

if filled_order:
    filled_amount = float(filled_order['amount'])
    print(f"Buy order filled at {buy_price}, placing STOP LOSS...")

    # ---------- Determine STOP LOSS ----------
    sl_price = buy_price -50  # 2% below buy price
    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])

    # If stop loss is below current market, use LIMIT; if above, use STOP_MARKET
    if sl_price <= current_market:
        # LIMIT stop-loss
        sl_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=sl_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT STOP LOSS at {sl_price}, order id: {sl_order['id']}")
    else:
        # STOP_MARKET stop-loss
        sl_order = exchange.create_order(
            symbol=symbol,
            type='stop_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': sl_price, 'reduceOnly': True}
        )
        print(f"Placed SELL STOP_MARKET STOP LOSS at {sl_price}, order id: {sl_order['id']}")


In [7]:
import time
import ccxt

symbol = "BTC/USDT"
amount = 0.001

# ---------- Fetch current market ----------
ticker = exchange.fetch_ticker(symbol)
market_price = float(ticker['last'])
print("Current market price:", market_price)

# ---------- Place a buy order BELOW market ----------
buy_price = market_price - 10  # example: $50 below current price
order = exchange.create_order(
    symbol=symbol,
    type='limit',
    side='buy',
    amount=amount,
    price=buy_price,
    params={'timeInForce': 'GTC'}
)
print(f"Placed BUY LIMIT at {buy_price}, order id: {order['id']}")

# ---------- Wait until filled (demo: fetch closed orders after 5 sec) ----------
time.sleep(5)
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break

if filled_order:
    filled_amount = float(filled_order['amount'])
    print(f"Buy order filled at {buy_price}, placing TP and SL...")

    # ---------- Determine Take Profit ----------
    tp_price = buy_price + 50  # 2% above buy
    # ---------- Determine Stop Loss ----------
    sl_price = buy_price - 50  # 2% below buy

    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])

    # --- PLACE STOP LOSS ---
    if sl_price >= current_market:
        # LIMIT stop-loss (below market)
        sl_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=sl_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT STOP LOSS at {sl_price}, order id: {sl_order['id']}")
    else:
        # STOP_MARKET stop-loss (above market)
        sl_order = exchange.create_order(
            symbol=symbol,
            type='stop_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': sl_price, 'reduceOnly': True}
        )
        print(f"Placed SELL STOP_MARKET STOP LOSS at {sl_price} 2, order id: {sl_order['id']}")

    # --- PLACE TAKE PROFIT ---
    if tp_price <= current_market:
        # LIMIT take-profit (below market, unusual)
        tp_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=tp_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT TP at {tp_price}, order id: {tp_order['id']}")
    else:
        # TAKE_PROFIT_MARKET (above market)
        tp_order = exchange.create_order(
            symbol=symbol,
            type='take_profit_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': tp_price, 'reduceOnly': True}
        )
        print(f"Placed SELL TAKE_PROFIT_MARKET TP at {tp_price} 2, order id: {tp_order['id']}")


Current market price: 107580.4


NotSupported: binance testnet/sandbox mode is not supported for futures anymore, please check the deprecation announcement https://t.me/ccxt_announcements/92 and consider using the demo trading instead.

In [74]:
closed_orders = exchange.fetch_closed_orders(symbol)
filled_order = None
for o in closed_orders:
    if float(o['average']) == buy_price and o['side'] == 'buy':
        filled_order = o
        break

if filled_order:
    filled_amount = float(filled_order['amount'])
    print(f"Buy order filled at {buy_price}, placing TP and SL...")

    # ---------- Determine Take Profit ----------
    tp_price = buy_price + 50  # 2% above buy
    # ---------- Determine Stop Loss ----------
    sl_price = buy_price - 50  # 2% below buy

    ticker = exchange.fetch_ticker(symbol)
    current_market = float(ticker['last'])

    # --- PLACE STOP LOSS ---
    if sl_price >= current_market:
        # LIMIT stop-loss (below market)
        sl_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=sl_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT STOP LOSS at {sl_price}, order id: {sl_order['id']}")
    else:
        # STOP_MARKET stop-loss (above market)
        sl_order = exchange.create_order(
            symbol=symbol,
            type='stop_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': sl_price, 'reduceOnly': True}
        )
        print(f"Placed SELL STOP_MARKET STOP LOSS at {sl_price}, order id: {sl_order['id']}")

    # --- PLACE TAKE PROFIT ---
    if tp_price <= current_market:
        # LIMIT take-profit (below market, unusual)
        tp_order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side='sell',
            amount=filled_amount,
            price=tp_price,
            params={'reduceOnly': True, 'timeInForce': 'GTC'}
        )
        print(f"Placed SELL LIMIT TP at {tp_price}, order id: {tp_order['id']}")
    else:
        # TAKE_PROFIT_MARKET (above market)
        tp_order = exchange.create_order(
            symbol=symbol,
            type='take_profit_market',
            side='sell',
            amount=filled_amount,
            params={'stopPrice': tp_price, 'reduceOnly': True}
        )
        print(f"Placed SELL TAKE_PROFIT_MARKET TP at {tp_price}, order id: {tp_order['id']}")


Buy order filled at 110693.0, placing TP and SL...
Placed SELL STOP_MARKET STOP LOSS at 110643.0, order id: 6310707048
Placed SELL TAKE_PROFIT_MARKET TP at 110743.0, order id: 6310709484


In [6]:
amount = 0.001
symbol = "BTC/USDT"
order = exchange.create_order(
    symbol=symbol,
    type='market',   # <---- market order
    side='buy',      # buy at current price
    amount=amount,
    params={'reduceOnly': False}  # optional
)

print("✅ Market BUY placed:", order)

NotSupported: binance testnet/sandbox mode is not supported for futures anymore, please check the deprecation announcement https://t.me/ccxt_announcements/92 and consider using the demo trading instead.